In [1]:
%matmodlab

Loading BokehJS ...

Populating the interactive namespace from matmodlab and bokeh


# Elastic Material

## Uniaxial Stress

Simulations to show uniaxial stress using two linear elastic models



In [2]:
models = {}
# setup the simulation
models['mps-1'] = MaterialPointSimulator('linear_elastic')

# set up the material, elastic model and elastic model implemented in python
parameters = {'K': 9.980040E+09, 'G': 3.750938E+09}
models['mps-1'].Material('elastic', parameters)

# define the steps
x, N = .01, 20
models['mps-1'].MixedStep(components=(1, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-1'].MixedStep(components=(2, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-1'].MixedStep(components=(1, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-1'].MixedStep(components=(0, 0, 0), descriptors='ESS', scale=x, frames=N)

# Duplicate the model and run
models['mps-2'] = MaterialPointSimulator('py_linear_elastic')
models['mps-2'].Material('pyelastic', parameters)
models['mps-2'].MixedStep(components=(1, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-2'].MixedStep(components=(2, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-2'].MixedStep(components=(1, 0, 0), descriptors='ESS', scale=x, frames=N)
models['mps-2'].MixedStep(components=(0, 0, 0), descriptors='ESS', scale=x, frames=N)

# check the difference
data_1 = np.array(models['mps-1'].get('E.XX', 'S.XX'))
data_2 = np.array(models['mps-2'].get('E.XX', 'S.XX'))

m = np.amax(data_1[:,1])
assert np.allclose(data_1[:,1]/m, data_2[:,1]/m)

J0_1 = models['mps-1'].material.J0
J0_2 = models['mps-2'].material.J0
assert np.allclose(J0_1, J0_2)

# Plastic Material

## Uniaxial Stress

In [3]:
# setup the simulation
mps = MaterialPointSimulator('plastic')

# set up the material
parameters = {'E': 10e6, 'Nu': .33, 'Y': 40e3}
mps.Material('plastic', parameters)

# define the steps
N = 100
mps.MixedStep(components=(.02, 0, 0), descriptors='ESS', frames=N)
mps.MixedStep(components=(0, 0, 0), descriptors='ESS', frames=N)

S = np.amax(mps.SDV.Mises)
assert (mps.material.parameters['Y'] - S) < 1.e-6
p = mps.plot('E.XX', 'S.XX')
show(p)

/Users/tjfulle/Development/matmodlab/matmodlab/mmd/simulator.py:305: UserWarning: Steps are run at time of their creation, the run method will be deprecated
  warnings.warn('Steps are run at time of their creation, '


# Polynomial Hyperelastic

In [4]:
E = 500.
Nu = .4999
C10 = E / (4. * (1. + Nu))
C10, C01 = C10 / 2., C10 / 2.
D1 = 6. * (1. - 2. * Nu) / E

mps = MaterialPointSimulator('polyhyper')

# set up the material - the two materials are identical
mps.Material('hyperelastic:polynomial', {'C10': C10, 'C01': C01, 'D1': D1})

mps.MixedStep(components=(.1,0,0), descriptors='ESS', frames=100)
mps.MixedStep(components=(0,0,0), descriptors='ESS', frames=100)

show(mps.plot('E.XX', 'S.XX'))

# Viscoelastic

In [5]:
temp = (75, 95)
time_f = 50
E, Nu = 500, .45

mps = MaterialPointSimulator('viscoelastic', initial_temperature=75)
parameters = [E, Nu]
prony_series =  np.array([[.35, 600.], [.15, 20.], [.25, 30.],
                          [.05, 40.], [.05, 50.], [.15, 60.]])
f = os.path.join(MAT_D, 'src/umat_neohooke.f90')
mat = mps.Material(UMAT, parameters, libname='umat_neohooke',
                   source_files=[f])

mat.Expansion(ISOTROPIC, [1.E-5])
mat.TRS(WLF, [75, 35, 50])
mat.Viscoelastic(PRONY, prony_series)
mps.MixedStep(components=(.1, 0., 0.), descriptors='ESS', increment=1.,
              temperature=75., frames=100)
mps.StrainRateStep(components=(0., 0., 0.), increment=50.,
                   temperature=95., frames=50)

show(mps.plot('Time', 'S.XX'))

Shear relaxation time term <=0, SETTING TO 1
Shear relaxation time term <=0, SETTING TO 1
Shear relaxation time term <=0, SETTING TO 1
Shear relaxation time term <=0, SETTING TO 1


# Thermal Expansion

In [6]:
consts = elas(E=500., Nu=.45)
K = consts['K']
G = consts['G']

mps = MaterialPointSimulator('expansion', initial_temperature=75)
parameters = [K, G]
mat = mps.Material('elastic', parameters)
mat.Expansion(ISOTROPIC, [1.E-5])
mps.MixedStep(components=(.1,0,0), descriptors='ESS',
              temperature=75., frames=10)
mps.StrainRateStep(components=(0,0,0), temperature=150., frames=10)
mps.run()
show(mps.plot('E.XX', 'S.XX'))